In [24]:
import json
import pickle
import traceback

from bs4 import BeautifulSoup
import requests



In [25]:
def get_all_chem_records():
    print("GET ALL RECORDS")
    url = 'https://en.wikipedia.org/w/api.php?action=query&list=embeddedin&eititle=Template:Chembox&format=json&eilimit=500'
    resp=requests.get(url)
    data=json.loads(resp.text)
    full_data=data['query']['embeddedin']
    while 'continue' in data:
        print(data['continue']['eicontinue'].split("|")[1],end="\r")
        resp=requests.get(url+"&eicontinue="+data['continue']['eicontinue'])
        data=json.loads(resp.text)
        full_data.extend(data['query']['embeddedin'])
    return full_data

def get_raw_wiki_data(full_data,book=None,max=None):
    all_data={} if book is None else book
    ini_len=len(all_data)
    l=len(full_data)
    title=" "*10
    if max is None:
        max=2*len(full_data)
    for i,page in enumerate(full_data):
        if page['ns'] in [1,2,3,4,5,6,10,11]:
            continue # skip namesapces https://en.wikipedia.org/wiki/Wikipedia:Namespace

        if len(all_data)-ini_len >= max:
            break
        pid=page['pageid']
        if pid in all_data:
            continue
        try:
            url = 'https://en.wikipedia.org/w/api.php?action=parse&format=json&pageid={}&prop=parsetree'.format(pid)
            resp=requests.get(url)
            data=json.loads(resp.text)
            d=data['parse']['parsetree']['*']
            #print(data.keys())
            ntitle=data['parse']['title']
            print(i,"/",l," "*len(title),end="\r")
            print(i,"/",l,ntitle,end="\r")
            title=ntitle
            soup=BeautifulSoup(d,)
            for comment in soup.findAll("comment"):
                comment.decompose()
            for comment in soup.findAll("ref"):
                comment.decompose()
            for t in soup.findAll(text=True):
                text = str(t).strip()
                t.replaceWith(text.lower())

            box=soup.find("title",string="chembox")
            if box is None:
                for t in soup.findAll("title"):
                    if t.text == "chembox":
                        box = t
                        break

            if box is None:
                box=soup.find("title",string="infobox chemical")
                if box is None:
                    for t in soup.findAll("title"):
                        if t.text == "infobox chemical":
                            box = t
                            break
            box = box.parent

            def dispatch_template(tl):
                parts=tl.findAll("part",recursive=False)

                if len(parts)<1:
                    return tl.find("title").text
                data={}
                for part in parts:
                    name=part.findAll("name",recursive=False)
                    assert len(name) == 1
                    value=part.findAll("value",recursive=False)
                    assert len(value) == 1
                    name=name[0]
                    value=value[0]

                    if name.text == "":
                        key=name.attrs["index"]
                    else:
                        key=name.text
                    if key.startswith("section"):
                        try:
                            key =value.find("template").find("title").text
                        except:
                            pass
                    #print(key)
                    valchildren = value.findChildren(recursive=False)
                    if len(valchildren) == 0:
                        if value.text == "":
                            continue
                        data[key]=value.text
                    else:
                        vc_data=[]
                        for vc in valchildren:
                            if vc.name == "template":
                                vc_data.append(dispatch_template(vc))
                            elif vc.name == "ext":
                                vc.decompose()
                                vc_data.append(value.text)
                            else:
                                raise NotImplementedError(vc)
                        data[key] = vc_data

                if list(data.keys()) == [str(i) for i in range(1,len(data)+1)]:
                    data=[data[str(i)] for i in range(1,len(data)+1)]
                return data
            data = dispatch_template(box)
            all_data[pid]={'data':data,'title':title}
        except:
            traceback.print_exc()
            print(soup.findAll("title"))
            print(page)
            break
    return all_data

In [26]:
try:
    with open("all_chem_records.pickle","rb") as f:
        all_chem_records = pickle.load(f)
except:
    with open("all_chem_records.pickle","w+b") as f:
        pickle.dump(get_all_chem_records(),f)
    with open("all_chem_records.pickle","rb") as f:
        all_chem_records=pickle.load(f)

try:
    with open("all_chem_raw_wiki_data.pickle","rb") as f:
        all_chem_raw_wiki_data = pickle.load(f)
except:
    all_chem_raw_wiki_data={}
all_chem_raw_wiki_data={}

l=len(all_chem_raw_wiki_data)
all_chem_raw_wiki_data=get_raw_wiki_data(all_chem_records,book=all_chem_raw_wiki_data,max=100)
while len(all_chem_raw_wiki_data)>l:
    break
    l=len(all_chem_raw_wiki_data)
    all_chem_raw_wiki_data=get_raw_wiki_data(all_chem_records,book=all_chem_raw_wiki_data,max=10)
    with open("all_chem_raw_wiki_data.pickle","w+b") as f:
        pickle.dump(all_chem_raw_wiki_data,f)
    with open("all_chem_raw_wiki_data.pickle","rb") as f:
        all_chem_raw_wiki_data=pickle.load(f)

In [106]:
import itertools
flatten = lambda l:list(itertools.chain.from_iterable(l))
import functools
import operator
flatten = lambda l: functools.reduce(operator.iconcat, l, [])
import numpy as np
flatten=lambda l: np.array(l).flatten().tolist()

all_keys=set()

ignore_keys={'imagefile',
             'imagefile1_ref', 'imagefiler2', 'imagename',
             'imagefilel1',
             'imagealt1',
             'imagealtr1',
             'imagenamer2',
             'imagesizel2',
             'imagecaption2',
             'imagefilel3',
             'imagesize3',
             'imagenamer3',
             'imagefiler3_ref',
             'imagefilel3_ref',
             'imagefile1', 'imagenamer1',
             'imagesizer2', 'imagealtl2', 'chemspiderid_ref', 'imagename2',
             'style',
             'imagealt2',
             'pronounce',
             'chembox hazards',
             'unii2_ref',
             'casno_ref',
             'imagefile3_ref', 'imagefile2', 'imagefile3', 'imagecaptionr1', 'imagefilel1_ref',
             'imagealt', 'imagealtl1', 'imagealtr2', 'imagecaption3', 'imagecaptionl1', 'imagefile2_ref',
             'imagefile_ref', 'imagefilel2', 'imagefilel2_ref', 'imagefiler1', 'imagefiler1_ref', 'imagefiler2_ref',
             'imagename1', 'imagename3', 'imagenamel1', 'imagenamel2', 'imagesize', 'imagesize1', 'imagesize2',
             'imagesizel1', 'imagesizer1', 'inchi1', 'inchikey1',
             'verifiedfields',
             'casno1_ref', 'chebi_ref', 'chembl_ref',
             'container_only',
             'stdinchi_ref', 'stdinchikey_ref',
              'verifiedimages', 'verifiedrevid', 'watchedfields',
             'drugbank_ref',
              'kegg_ref',
              'unii1_ref', 'unii_ref','unii2_ref',
             'reference',
             #'nfpa_ref',
             #'pel',
             #'rel',
             #'idlh'
             }

to_expand={'data',
           #'chembox hazards',
           'chembox identifiers',
           }
to_save={'title':'title',
         'name':'name','systematicname':'systematicname',
         'pin':'pin',
         'iupacname':'iupacname',
         'othernames':'othernames',
         'casno':'casno',
         'ghspictograms':'ghspictograms',
         'hphrases':'hphrases',
         'pphrases':'pphrases',
         'iupacnames':'iupacnames',
         'abbreviations':'abbreviations',
         
         'stdinchikey':'stdinchikey',
         'stdinchi':'stdinchi',
         'inchi': 'inchi',
         'inchi2':'inchi2',
         'inchikey':'inchikey',
         'inchikey2':'inchikey2',
         'smiles':'smiles','smiles_comment':'smiles_comment',
         'smiles1':'smiles1', 'smiles1_comment':'smiles1_comment',
         'smiles2':'smiles2', 'smiles2_comment':'smiles2_comment',
         'smiles3':'smiles3', 'smiles3_comment':'smiles3_comment',
         
         'casno1':'casno1','casno1_comment':'casno1_comment',
         'casno2':'casno2','casno2_comment':'casno2_comment',
         'chebi':'chebi',
         'chembl':'chembl',
         'chemspiderid':'chemspiderid',
         'drugbank':'drugbank', 
         'kegg':'kegg',
         'pubchem':'pubchem','pubchem_comment':'pubchem_comment',
         'pubchem1':'pubchem1', 'pubchem1_comment':'pubchem1_comment',
         'pubchem2':'pubchem2', 'pubchem2_comment':'pubchem2_comment',
         'pubchem3':'pubchem3', 'pubchem3_comment':'pubchem3_comment',
         'unii':'unii','unii_comment':'unii_comment',
         'unii1':'unii1', 'unii1_comment':'unii1_comment',
         'unii2':'unii2', 'unii2_comment':'unii2_comment',
         '3dmet':'3dmet',
         'ec_number':'ec_number',
         'rtecs':'rtecs',
         'beilstein':'beilstein',
         'einecs':'einecs',
         
         'meshname':'meshname',
         
         'unnumber':'unnumber',
         
         
         #'ld50':'ld50'
         }
as_list={'systematicname','ghspictograms','hphrases','pphrases','othernames'}


known_keys = ignore_keys.union(to_expand).union(to_save.keys())

extracted_data={}

error_extract=set()

def flatten(l):
    rl=[]
    for i in l:
        if isinstance(i,list):
            rl+=flatten(i)
        else:
            rl.append(i)
    return rl

def unlist(data,):
    if not isinstance(data,list):
        return data
    assert len(data)==1,"len not 1({}) {}".format(len(data),data)
    return data[0]

def tolist(data):
    if not isinstance(data,list):
            return [data]
    return flatten(data)

def expand(data,pid):
    all_keys.update(data.keys())
    for k,v in data.items():
        try:
            if k in ignore_keys:
                continue
            if k in as_list:
                v=tolist(v)
            else:
                v=unlist(v)
            if k in to_save:
                assert not isinstance(v,dict), "isdict"
                assert k not in extracted_data[pid],"already there"
                extracted_data[pid][k]=v
            if k in to_expand:
                expand(v,pid)
        except Exception as e:
            print(k,"<--",end=" ")
            raise e


for k,v in all_chem_raw_wiki_data.items():
    try:
        extracted_data[k]={}
        expand(v,k)
    except Exception as e:
        error_extract.add(k)
        print(k)
        print(v)
        print("ER",e)
        print(extracted_data[k])
        del extracted_data[k]
        break
    

unknown_keys=all_keys-known_keys
print(error_extract)
print(sorted(unknown_keys))

#extracted_data

set()
['chembox explosive', 'chembox pharmacology', 'chembox properties', 'chembox related', 'chembox structure', 'chembox thermochemistry', 'gmelin', 'iuphar_ligand']


In [107]:
p2p=[
    'title',
    #'casno','iupacnames',
    'meshname'
]
for k,v in extracted_data.items():
    print("\n",k,end = "   ")
    for sk,sv in v.items():
        if sk in p2p:
            print(sk,":",sv,end=", ")
    


 897   title : Arsenic, 
 1365   meshname : ammonia, title : Ammonia, 
 1778   title : Acetylene, 
 1800   title : Adenosine triphosphate, 
 2268   title : Chemistry of ascorbic acid, 
 2752   title : Aspartame, 
 3252   meshname : amygdalin, title : Amygdalin, 
 3370   meshname : elbor, title : Boron nitride, 
 3378   title : Beryllium, 
 3755   title : Boron, 
 3756   title : Bromine, 
 3757   title : Barium, 
 3982   title : Bicarbonate, 
 4106   meshname : benzoic+acid, title : Benzoic acid, 
 4485   title : Bakelite, 
 5667   title : Chlorine, 
 5668   title : Calcium, 
 5672   title : Cadmium, 
 5879   title : Caesium, 
 5906   meshname : carbon+dioxide, title : Carbon dioxide, 
 5910   title : Cyanide, 
 5912   title : Carbonate, 
 6016   meshname : cytosine, title : Cytosine, 
 6136   meshname : carbon+monoxide, title : Carbon monoxide, 
 6437   title : Cholesterol, 
 6911   title : Cellulose, 
 6988   meshname : cyclic+amp, title : Cyclic adenosine monophosphate, 
 7499   tit

{897: {'data': {'container_only': 'yes',
   'chembox hazards': [{'ghspictograms': ['ghs06', 'ghs08', 'ghs09'],
     'ghssignalword': 'danger',
     'hphrases': [['301', '331', '350', '410'], 'h-phrases301331350410']}]},
  'title': 'Arsenic'},
 1365: {'data': {'watchedfields': 'changed',
   'verifiedrevid': '4453674488',
   'imagefilel2': 'ammonia-3d-balls-a.png',
   'imagenamel2': 'ball-and-stick model of the ammonia molecule',
   'imagefiler2': 'ammonia-3d-vdw.png',
   'imagenamer2': 'space-filling model of the ammonia molecule',
   'imagefile1': 'ammonia-2d.svg',
   'imagesize1': '150px',
   'imagename1': 'stereo structural formula of the ammonia molecule',
   'iupacnames': ['ammonia<br /> trihydridonitrogen<br />\nnitrogen trihydride'],
   'othernames': 'azane (only intended for use in naming derivatives of ammonia)<br /> hydrogen nitride<br />\nr-717 (refrigerant)<br />\nr717 (refrigerant alternative spelling)',
   'chembox identifiers': [{'inchi': '1/h3n/h1h3',
     'inchikey': 'q